In [57]:
import sys
!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import pprint as pp
from collections import OrderedDict
import requests

# File to Load (Remember to Change These)
file_to_load = "schoolShootingData.csv"

# Read Purchasing File and store into Pandas data frame
schoolData = pd.read_csv(file_to_load, skiprows=[0], encoding="utf-8-sig")
geolocator = Nominatim(user_agent="classProject",timeout=5)

In [58]:
type(schoolData)
schoolData.head(2)

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Firearm Type,Number of Shooters,Shooter Name,Shooter Age,Shooter Gender,Shooter Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N)
0,1/5/1970,Hine Junior High School,Washington,DC,3.0,1,0.0,1,Male,Student,...,Handgun,1.0,Minor,15.0,M,Unknown,Unknown,Student,Y,N
1,1/5/1970,Sousa Junior High,Washington,DC,3.0,0,1.0,1,Male,Student,...,Handgun,1.0,Minor,NaN,M,Unknown,Unknown,Student,N,N


In [53]:
# This times out. Look at bootom one for the exact details
locations = []
for index, row in schoolData.iterrows():
    address = row['School']+","+ row['City']+","+row['State']
    location = geolocator.geocode(address)
    try:
        if hasattr(location, 'latitude'):
            locations.append([ address , location.latitude, location.longitude])
        else:
            response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+address)
            print(resp_json_payload['results'][0]['geometry']['location'])
            locations.append([address,"unknown","unknown"   ]) 
    except Exception as e:   
            locations.append([address,"unknown","unknown"]) 
            print(str(e))            


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [46]:
type(locations)

list

In [49]:
len(schoolData)

1373

In [50]:
locationsDF = pd.DataFrame(locations)

In [51]:
locationsDF.to_csv ('latLong.csv')


In [67]:
apiKey = 'YOURKEYHERE'
locationsGoogle = []
for index, row in schoolData.iterrows():
    try:
        address = row['School']+","+ row['City']+","+row['State']
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+address+"&key="+apiKey)
        #print('https://maps.googleapis.com/maps/api/geocode/json?address='+address+"&key="+apiKey)
        resp_json_payload = response.json()
        #print(resp_json_payload['results'][0]['geometry']['location'])
        locationsGoogle.append([address,resp_json_payload['results'][0]['geometry']['location']])
    except Exception as e:   
            locationsGoogle.append([address,"{unknown,unknown}"]) 
            print(str(e)) 
    

list index out of range


In [68]:
len(locationsGoogle)

1373

In [69]:
locationsDFGoogle = pd.DataFrame(locationsGoogle)

In [71]:
locationsDFGoogle.to_csv ('latLongGoogle.csv')


In [73]:
len(locationsDFGoogle)


1373

In [74]:
len(schoolData)

1373

In [76]:
locationsDFGoogle = locationsDFGoogle.reset_index(drop=True)
schoolData = schoolData.reset_index(drop=True)
schoolData = schoolData.join(locationsDFGoogle)

In [77]:
schoolData.head()

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Shooter Name,Shooter Age,Shooter Gender,Shooter Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N),0,1
0,1/5/1970,Hine Junior High School,Washington,DC,3.0,1,0.0,1,Male,Student,...,Minor,15.0,M,Unknown,Unknown,Student,Y,N,"Hine Junior High School,Washington,DC","{'lat': 38.8923472, 'lng': -76.97896759999999}"
1,1/5/1970,Sousa Junior High,Washington,DC,3.0,0,1.0,1,Male,Student,...,Minor,NaN,M,Unknown,Unknown,Student,N,N,"Sousa Junior High,Washington,DC","{'lat': 38.8839745, 'lng': -76.9531487}"
2,1/5/1970,Unnamed High School,Washington,DC,2.0,0,0.0,0,No Victims,No Victims,...,Unknown,NaN,M,Unknown,Unknown,Student,Y,N,"Unnamed High School,Washington,DC","{'lat': 39.3201044, 'lng': -77.349205}"
3,2/6/1970,John F. Kennedy High School,Cleveland,OH,2.0,0,1.0,1,Male,Student,...,Gertis J. Perry,18.0,M,Unknown,Not Hispanic or Latino,Student,N,N,"John F. Kennedy High School,Cleveland,OH","{'lat': 41.4494718, 'lng': -81.5607}"
4,3/23/1970,David Starr Jordan High School,Long Beach,CA,2.0,0,2.0,2,Male,Multiple Victims,...,"Obidde Cowart, Lavaughn Hunter",NaN,M,Unknown,Not Hispanic or Latino,Other Staff,Y,N,"David Starr Jordan High School,Long Beach,CA","{'lat': 33.8722207, 'lng': -118.1840179}"


In [84]:
schoolData = schoolData.rename(columns={0: 'address', 1: 'location'})

In [85]:
schoolData.head()

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Shooter Name,Shooter Age,Shooter Gender,Shooter Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N),address,location
0,1/5/1970,Hine Junior High School,Washington,DC,3.0,1,0.0,1,Male,Student,...,Minor,15.0,M,Unknown,Unknown,Student,Y,N,"Hine Junior High School,Washington,DC","{'lat': 38.8923472, 'lng': -76.97896759999999}"
1,1/5/1970,Sousa Junior High,Washington,DC,3.0,0,1.0,1,Male,Student,...,Minor,NaN,M,Unknown,Unknown,Student,N,N,"Sousa Junior High,Washington,DC","{'lat': 38.8839745, 'lng': -76.9531487}"
2,1/5/1970,Unnamed High School,Washington,DC,2.0,0,0.0,0,No Victims,No Victims,...,Unknown,NaN,M,Unknown,Unknown,Student,Y,N,"Unnamed High School,Washington,DC","{'lat': 39.3201044, 'lng': -77.349205}"
3,2/6/1970,John F. Kennedy High School,Cleveland,OH,2.0,0,1.0,1,Male,Student,...,Gertis J. Perry,18.0,M,Unknown,Not Hispanic or Latino,Student,N,N,"John F. Kennedy High School,Cleveland,OH","{'lat': 41.4494718, 'lng': -81.5607}"
4,3/23/1970,David Starr Jordan High School,Long Beach,CA,2.0,0,2.0,2,Male,Multiple Victims,...,"Obidde Cowart, Lavaughn Hunter",NaN,M,Unknown,Not Hispanic or Latino,Other Staff,Y,N,"David Starr Jordan High School,Long Beach,CA","{'lat': 33.8722207, 'lng': -118.1840179}"


In [87]:
schoolData.to_csv ('udpatedCsvWithLocation.csv')